In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_376899/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/16 03:36:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/16 03:36:10 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for the Drivers table
drivers_data = {
    'driver_id': [10, 8, 5, 7, 4, 1, 6],
    'join_date': ['2019-12-10', '2020-01-13', '2020-02-16', '2020-03-08', '2020-05-17', '2020-10-24', '2021-01-05']
}

# Sample data for the Rides table
rides_data = {
    'ride_id': [6, 1, 10, 19, 3, 13, 7, 17, 20, 5, 2, 11, 15, 12, 14],
    'user_id': [75, 54, 63, 39, 41, 52, 69, 70, 81, 57, 42, 68, 32, 11, 18],
    'requested_at': ['2019-12-09', '2020-02-09', '2020-03-04', '2020-04-06', '2020-06-03', '2020-06-22', '2020-07-16', '2020-08-25', '2020-11-02', '2020-11-09', '2020-12-09', '2021-01-11', '2021-01-17', '2021-01-19', '2021-01-27']
}

# Sample data for the AcceptedRides table
accepted_rides_data = {
    'ride_id': [10, 13, 7, 17, 20, 5, 2, 11, 15, 12, 14],
    'driver_id': [10, 10, 8, 7, 1, 7, 4, 8, 8, 8, 1],
    'ride_distance': [63, 73, 100, 119, 121, 42, 6, 37, 108, 38, 90],
    'ride_duration': [38, 96, 28, 68, 92, 101, 38, 43, 82, 34, 74]
}

# Convert data to DataFrames
drivers_df = pd.DataFrame(drivers_data)
rides_df = pd.DataFrame(rides_data)
accepted_rides_df = pd.DataFrame(accepted_rides_data)

# Convert 'join_date' and 'requested_at' columns to datetime
drivers_df['join_date'] = pd.to_datetime(drivers_df['join_date'])
rides_df['requested_at'] = pd.to_datetime(rides_df['requested_at'])


df_person = spark.createDataFrame(drivers_df)
df_person.createOrReplaceTempView("Drivers")

df_person = spark.createDataFrame(rides_df)
df_person.createOrReplaceTempView("Rides")

df_person = spark.createDataFrame(accepted_rides_df)
df_person.createOrReplaceTempView("AcceptedRides")



In [4]:
drivers_df

,driver_id,join_date
0,10,2019-12-10
1,8,2020-01-13
2,5,2020-02-16
3,7,2020-03-08
4,4,2020-05-17
5,1,2020-10-24
6,6,2021-01-05


In [5]:
rides_df

,ride_id,user_id,requested_at
0,6,75,2019-12-09
1,1,54,2020-02-09
2,10,63,2020-03-04
3,19,39,2020-04-06
4,3,41,2020-06-03
5,13,52,2020-06-22
6,7,69,2020-07-16
7,17,70,2020-08-25
8,20,81,2020-11-02
9,5,57,2020-11-09


In [6]:
accepted_rides_df

,ride_id,driver_id,ride_distance,ride_duration
0,10,10,63,38
1,13,10,73,96
2,7,8,100,28
3,17,7,119,68
4,20,1,121,92
5,5,7,42,101
6,2,4,6,38
7,11,8,37,43
8,15,8,108,82
9,12,8,38,34


In [52]:
query = """
    with drivers_with_year_month as (
        select
            d.*,
            year(d.join_date) as year,
            month(d.join_date) as month
        from Drivers d
    ),
    
    driver_accepted as (
        select
            distinct
            year,
            month,
            count(*) over (partition by year, month) as driver_accept
        from (
            select
                distinct
                year(r.requested_at) as year,
                month(r.requested_at) as month,
                a.driver_id
            from 
                AcceptedRides a
                left join Rides r on r.ride_id = a.ride_id
            order by year, month
        )
    ),
    
    months_and_years as (
        select
            d.year,
            t.month
        from
            (select explode(sequence(1, 12)) as month) as t
            cross join (select distinct year from drivers_with_year_month) as d
    ),
    
    active_drivers as (
        select
            my.year,
            my.month,
            sum(
                case
                    when d.driver_id is null then 0
                    else 1
                end
            ) over (order by my.year asc, my.month asc) as active_drivers
        from
            months_and_years my left join drivers_with_year_month d on my.year = d.year and my.month = d.month
    )
    
    select 
        a.year,
        a.month,
        case
            when a.active_drivers = 0 then 0.00
            else round(ifnull(d.driver_accept, 0) / a.active_drivers * 100, 2)
        end as working_percentage
    from 
        active_drivers a
        left join driver_accepted d on a.year = d.year and a.month = d.month
    where a.year = 2020
    
"""

In [53]:
spark.sql(query).show(50)

24/03/16 04:30:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 04:30:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 04:30:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 04:30:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 04:30:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 04:30:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 0

+----+-----+------------------+
|year|month|working_percentage|
+----+-----+------------------+
|2020|    1|               0.0|
|2020|    2|               0.0|
|2020|    3|              25.0|
|2020|    4|               0.0|
|2020|    5|               0.0|
|2020|    6|              20.0|
|2020|    7|              20.0|
|2020|    8|              20.0|
|2020|    9|               0.0|
|2020|   10|               0.0|
|2020|   11|             33.33|
|2020|   12|             16.67|
+----+-----+------------------+

